In [ ]:
# importing required libraries

import random as rand
import pygame

Description of the algorithm used to build the random maze generator

• Treat the grid's cells as vertices (labelled from 0 to n-1)
• Two vertices are connected by an edge if:
    i. They are next to each other
    ii. The adjoining wall is absent
• Continue adding the edges, i.e. removing walls, till vertex 0 (entry point) is connected to vertex n-1 (exit point)
• Do not add edges within the same connected components, construct a minimum spanning tree with n-1 edges

Pseudocode:

MazeGenerator(G, E):
    Input: A grid, G, consisting of n cells and a set, E, of m “walls,” each of which divides two cells, x and y, so that the walls in E initially separate and isolate all cells in G.
    Output: A subset, R of E, such that adding the edges in R to E (i.e., removing the walls present in G using the information from R) creates a maze 

    while R has less than n-1 edges do 
        Choose an edge, (x, y), in E uniformly at random    
        if find(x) ≠ find(y) then do
            union(find(x), find(y))
            Add the edge (x, y) to R

    return R

In [ ]:
# Code for Maze Generator Using PyGame

pygame.init()

# Set up display
width, height = 700,700
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Maze Generator")

# Set background color
background_color = (255, 255, 255)  # White
screen.fill(background_color)

# Draw a 20*20 grid, construct all walls (black lines)
for i in range(21):
    pygame.draw.line(screen, (0, 0, 0), (0+50, 30*i+50), (600+50, 30*i+50), 1)
    pygame.draw.line(screen, (0, 0, 0), (30*i+50, 50), (30*i+50,600+50), 1)

#Start and End points (erasing walls using white lines)
pygame.draw.line(screen, (255, 255, 255), (50,50), (50,80), 2)
pygame.draw.line(screen, (255, 255, 255), (650,620), (650,650), 2)

# Initializing DSU data structure and edge count
d = DSU(400)
edge_count= 0

# Do this until we obtain a spanning tree
while (edge_count<399):
    vert = lst[rand.randint(0,399)]    #randomly choosing a vertex
    nbr = (-1,-1)                #default neighbour

    if (vert[0]==65):                 #left vertical line
        if (vert[1]==65):               #top-left corner
            y_off=rand.randint(0,1)
        elif (vert[1]==635):             #bottom-left corner
            y_off=rand.randint(-1,0)
        else:
            y_off = rand.randint(-1,1)
        if (y_off==0):
            x_off=1
        else:
            x_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30) 

    elif (vert[0]==635):                #right vertical line
        if (vert[1]==65):               #top-right corner
            y_off=rand.randint(0,1)
        elif (vert[1]==635):             #bottom-right corner
            y_off=rand.randint(-1,0)
        else:
            y_off = rand.randint(-1,1)
        if (y_off==0):
            x_off=-1
        else:
            x_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30) 

    elif (vert[1]==65):                     #top horizontal line
        x_off = rand.randint(-1,1)           #all corner points covered, hence no need for larger if-else now onwards
        if (x_off==0):
            y_off=1
        else:
            y_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30)    

    elif (vert[1]==635):                     #bottom horizontal line
        x_off = rand.randint(-1,1)          
        if (x_off==0):
            y_off=-1
        else:
            y_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30) 

    else:                                    #all middle points
        x_off = rand.randint(-1,1)
        if (x_off==0):
            l = [-1,1]   
            y_off=rand.choice(l)   #either move-up or move-down
        else:
            y_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30) 

    #code reached here, now decide whether to add edge or not

    vertex1 = dct[vert]
    vertex2 = dct[nbr]

    if (d.is_connected(vertex1,vertex2)==False):
        d.union_(vertex1,vertex2)
        edge_count+=1
        if (vert[0]==nbr[0]):
            x1 = vert[0]-15
            x2 = vert[0]+15
            y = (vert[1]+nbr[1])/2
            pygame.draw.line(screen, (255, 255, 255), (x1,y), (x2,y), 2)
        else:
            x = (vert[0]+nbr[0])/2
            y1 = vert[1]-15
            y2 = vert[1]+15
            pygame.draw.line(screen, (255, 255, 255), (x,y1), (x,y2), 2)            

pygame.display.flip()

# ----- Uncomment the following code block if you wish to save the maze in a .jpg file -----

# Capture the screen surface
screenshot = pygame.Surface((width, height))
screenshot.blit(screen, (0, 0))

# Save the surface as a .jpg file
pygame.image.save(screenshot, 'maze.jpg')

# # Main loop
# running = True
# while running:
#     for event in pygame.event.get():
#         if event.type == pygame.QUIT:
#             running = False

# pygame.quit()

In [ ]:
# Code for Maze Solver Using DFS + Backtracking

# Function to get valid neighbors for DFS traversal
def get_neighbors(vertex_num):
    x, y = lst[vertex_num]
    neighbors = []
    
    # Check all 4 directions (up, down, left, right)
    directions = [(0, -30), (0, 30), (-30, 0), (30, 0)]
    
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        new_pos = (new_x, new_y)
        
        # Check if new position is within grid bounds and exists in our mapping
        if new_pos in dct:
            neighbor_vertex = dct[new_pos]
            # Check if vertices are connected (no wall between them)
            if d.is_connected(vertex_num, neighbor_vertex):
                neighbors.append(neighbor_vertex)
    
    return neighbors

# DFS function with backtracking
def dfs_solve(start, end, visited, path):
    visited.add(start)
    path.append(start)
    
    # If we reached the end, return True
    if start == end:
        return True
    
    # Explore neighbors
    for neighbor in get_neighbors(start):
        if neighbor not in visited:
            if dfs_solve(neighbor, end, visited, path):
                return True
    
    # Backtrack if no solution found through this path
    path.pop()
    return False

# Solve the maze using DFS
start_vertex = 0    # Top-left corner (entry point)
end_vertex = 399    # Bottom-right corner (exit point)
visited = set()
solution_path = []

print("Solving maze using DFS + Backtracking...")
if dfs_solve(start_vertex, end_vertex, visited, solution_path):
    print(f"Solution found! Path length: {len(solution_path)}")
    # Ensure pygame display is initialized
    if not pygame.get_init():
        pygame.init()
        screen = pygame.display.set_mode((width, height))
        pygame.display.set_caption("Maze Solver")
        # Redraw the maze here if needed
    
    # Draw the solution path in red
    for i in range(len(solution_path) - 1):
        current_pos = lst[solution_path[i]]
        next_pos = lst[solution_path[i + 1]]
        
        # Draw red line between consecutive vertices in the solution path
        pygame.draw.line(screen, (255, 0, 0), current_pos, next_pos, 3)
        
    # Update display
    pygame.display.flip()
    
    # Save the solved maze
    solved_screenshot = pygame.Surface((width, height))
    solved_screenshot.blit(screen, (0, 0))
    pygame.image.save(solved_screenshot, 'maze_solved.jpg')
    print("Solved maze saved as 'maze_solved.jpg'")
    
    # Keep the window open to show the solved maze
    print("Press the close button to exit...")
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.quit()
    
else:
    print("No solution found!")


Solving maze using DFS + Backtracking...
Solution found! Path length: 57
Solved maze saved as 'maze_solved.jpg'
Press the close button to exit...
